In [1]:
import pandas as pd
from pprint import pprint
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import re

In [22]:

col_multival = set()
non_list_cols =  ['season', 'aname', 'title', 'link',
                  'slot', 'day', 'startTime', "endTime",
                  "episodes", "episodesRev",
                  "synopsis", "station",
                  "date", "ave", "percentage", "drama_id"]

role2group = {'actor': 'cast',
              'producer': 'prod',
              'chiefProducer': 'prod',
              'director': 'prod',
              'script': 'prod',
              'original': 'prod',
              'themeSong': 'prod',
              'sungInDrama': 'prod',
              'inDramaSong': 'prod',
              'endingMusic': 'prod',
              'openingMusic': 'prod',
              'themeMusic': 'prod',
              'imageMusic': 'prod',
              'titleMusic': 'prod'}

def contains_katakana(text):
    katakana_pattern = re.compile('[\u30A1-\u30FA\u30FC]')
    return bool(katakana_pattern.search(text))

def parse_record(index, record):
    parsed_keys = set()
    record_dict = {}

    for key, value in zip(record.index, record):
        if isinstance(value, str):
            value = value.strip()

        if key in ['MATCHED','TYPE', 'DISTANCE']: # 'sungInDrama',
            continue

        elif key == "synopsis":
                record_dict['synopsis'] = value
        elif key in ['original', 'themeSong', 'sungInDrama',
                     'inDramaSong', 'endingMusic', 'openingMusic',
                     'themeMusic', 'imageMusic', 'titleMusic']:
            if isinstance(value, str):
                value_to_use = None
                if "," in value or "『" in value or "「" in value:
                    delimiters = ",", "×", "＆","『", "「"
                    regex_pattern = '|'.join(map(re.escape, delimiters))
                    parts = [e for e in re.split(regex_pattern, value) if e != '']
                    if len(parts) == 1:
                        value_to_use = [value]
                    else:
                        value_to_use = parts[0:-1]
                else:
                    value_to_use = [value]
                assert value_to_use is not None and type(value_to_use) == list

                new_value_lis = []
                for one_value in value_to_use:
                    if "・" in one_value:
                        if contains_katakana(one_value):
                            new_value_lis.append(one_value)
                        else:
                            for composed in one_value.split("・"):
                                new_value_lis.append(composed)
                    else:
                        new_value_lis.append(one_value)
                value_to_use = new_value_lis

                if record.aname == "jotei":
                    print(value)
                    print(value_to_use)

                record_dict[key.strip()] = value_to_use
        else:
            parsed_keys.add(key)
            if isinstance(value, str) and ',' in value:
                col_multival.add(key.strip())
                value = value.split(',')
                new_value_lis = []
                for one_val in value:

                    if "・" in one_val:
                        if contains_katakana(one_val):

                            new_value_lis.append(one_val)
                        else:
                            print(one_val)
                            assert 1==2
                            assert one_val.count("・") == 1
                            for composed in one_val.split("・"):
                                new_value_lis.append(composed)
                    else:
                        new_value_lis.append(one_val)
                value = new_value_lis
            record_dict[key.strip()] = value
    record_dict['drama_id'] = index


    to_df_dic_lis = []
    for col, values in record_dict.items():
        if col not in non_list_cols:
            if values != values or values == '':
                values = [np.nan]
            elif type(values) == str:
                values = [values]
            for value in values:
                one_value_dic = {}
                if value != value:
                    continue
                try:
                    one_value_dic['attr_name'] = role2group[col]
                except:
                    print(col)
                    assert 1==2
                one_value_dic['attr_value'] = value
                for other_col_name in non_list_cols:
                    one_value_dic[other_col_name] = record_dict[other_col_name]
                to_df_dic_lis.append(one_value_dic)
    return pd.DataFrame(to_df_dic_lis)

def csv_to_dataframe(file_path):
    artv = pd.read_csv(file_path)
    print(artv.shape)
    parsed_records = [parse_record(index, row)
                      for index, row in tqdm(artv.iterrows(), total=len(artv))]
    return pd.concat(parsed_records)

In [23]:

df = csv_to_dataframe("artv/merged2.csv")
df['drama_id'] = df['drama_id'].astype(str)
df['aname'] = df['aname'].astype(str)
df['title'] = df['title'].astype(str)
df['season'] = df['season'].astype(str)

df['drama_id'] = df['season'] + "_" + df['aname'] + "_" + df['title']

import unicodedata
def to_halfwidth(s):
    return unicodedata.normalize('NFKC', s)
df['attr_value'] = df['attr_value'].apply(to_halfwidth)
df['attr_value'] = df['attr_value'].str.strip()

print(df.columns)
print(df.shape)
# print(df.shape)
# df.head() (19923, 18)

(1036, 32)


  0%|          | 0/1036 [00:00<?, ?it/s]

 17%|█▋        | 178/1036 [00:00<00:00, 1776.56it/s]

倉科遼,和気一作「女帝」
['倉科遼', '和気一作']
ナナムジカ,『彼方』
['ナナムジカ']


100%|██████████| 1036/1036 [00:00<00:00, 1776.52it/s]


Index(['attr_name', 'attr_value', 'season', 'aname', 'title', 'link', 'slot',
       'day', 'startTime', 'endTime', 'episodes', 'episodesRev', 'synopsis',
       'station', 'date', 'ave', 'percentage', 'drama_id'],
      dtype='object')
(20270, 18)


In [24]:
df

,attr_name,attr_value,season,aname,title,link,slot,day,startTime,endTime,episodes,episodesRev,synopsis,station,date,ave,percentage,drama_id
0,cast,松たか子,301,itsumo,いつもふたりで,http://www.fujitv.co.jp/itsumo/,月曜21：00〜21：54 （全11回）,Mon,21:00,21:54,11,11,有名小説家になりたいという夢を持つ谷町瑞穂（松たか子）は応募した新人文学賞を受賞し、遂にその...,フジテレビ,"[1/6, 1/13, 1/20, 1/27, 2/3, 2/10, 2/17, 2/24,...",16.17,"[18.20, 16.40, 15.30, 17.30, 16.00, 15.10, 14....",301_itsumo_いつもふたりで
1,cast,坂口憲二,301,itsumo,いつもふたりで,http://www.fujitv.co.jp/itsumo/,月曜21：00〜21：54 （全11回）,Mon,21:00,21:54,11,11,有名小説家になりたいという夢を持つ谷町瑞穂（松たか子）は応募した新人文学賞を受賞し、遂にその...,フジテレビ,"[1/6, 1/13, 1/20, 1/27, 2/3, 2/10, 2/17, 2/24,...",16.17,"[18.20, 16.40, 15.30, 17.30, 16.00, 15.10, 14....",301_itsumo_いつもふたりで
2,cast,柏原崇,301,itsumo,いつもふたりで,http://www.fujitv.co.jp/itsumo/,月曜21：00〜21：54 （全11回）,Mon,21:00,21:54,11,11,有名小説家になりたいという夢を持つ谷町瑞穂（松たか子）は応募した新人文学賞を受賞し、遂にその...,フジテレビ,"[1/6, 1/13, 1/20, 1/27, 2/3, 2/10, 2/17, 2/24,...",16.17,"[18.20, 16.40, 15.30, 17.30, 16.00, 15.10, 14....",301_itsumo_いつもふたりで
3,cast,長谷川京子,301,itsumo,いつもふたりで,http://www.fujitv.co.jp/itsumo/,月曜21：00〜21：54 （全11回）,Mon,21:00,21:54,11,11,有名小説家になりたいという夢を持つ谷町瑞穂（松たか子）は応募した新人文学賞を受賞し、遂にその...,フジテレビ,"[1/6, 1/13, 1/20, 1/27, 2/3, 2/10, 2/17, 2/24,...",16.17,"[18.20, 16.40, 15.30, 17.30, 16.00, 15.10, 14....",301_itsumo_いつもふたりで
4,cast,平山綾,301,itsumo,いつもふたりで,http://www.fujitv.co.jp/itsumo/,月曜21：00〜21：54 （全11回）,Mon,21:00,21:54,11,11,有名小説家になりたいという夢を持つ谷町瑞穂（松たか子）は応募した新人文学賞を受賞し、遂にその...,フジテレビ,"[1/6, 1/13, 1/20, 1/27, 2/3, 2/10, 2/17, 2/24,...",16.17,"[18.20, 16.40, 15.30, 17.30, 16.00, 15.10, 14....",301_itsumo_いつもふたりで
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,prod,清弘誠,310,suekko,末っ子長男姉三人,http://www.tbs.co.jp/suekko/,日曜21：00〜21：54 （全10回）,Sun,21:00,21:54,10,10,NO_SYNOPSIS,TBS,"[10/12, 10/19, 10/26, 11/2, 11/16, 11/23, 11/3...",14.39,"[18.60, 16.40, 12.80, 13.80, 13.20, 11.30, 12....",310_suekko_末っ子長男姉三人
9,prod,金子文紀,310,suekko,末っ子長男姉三人,http://www.tbs.co.jp/suekko/,日曜21：00〜21：54 （全10回）,Sun,21:00,21:54,10,10,NO_SYNOPSIS,TBS,"[10/12, 10/19, 10/26, 11/2, 11/16, 11/23, 11/3...",14.39,"[18.60, 16.40, 12.80, 13.80, 13.20, 11.30, 12....",310_suekko_末っ子長男姉三人
10,prod,高成麻畝子,310,suekko,末っ子長男姉三人,http://www.tbs.co.jp/suekko/,日曜21：00〜21：54 （全10回）,Sun,21:00,21:54,10,10,NO_SYNOPSIS,TBS,"[10/12, 10/19, 10/26, 11/2, 11/16, 11/23, 11/3...",14.39,"[18.60, 16.40, 12.80, 13.80, 13.20, 11.30, 12....",310_suekko_末っ子長男姉三人
11,prod,吉田紀子,310,suekko,末っ子長男姉三人,http://www.tbs.co.jp/suekko/,日曜21：00〜21：54 （全10回）,Sun,21:00,21:54,10,10,NO_SYNOPSIS,TBS,"[10/12, 10/19, 10/26, 11/2, 11/16, 11/23, 11/3...",14.39,"[18.60, 16.40, 12.80, 13.80, 13.20, 11.30, 12....",310_suekko_末っ子長男姉三人


In [17]:

df

,attr_name,attr_value,season,aname,title,link,slot,day,startTime,endTime,episodes,episodesRev,synopsis,station,date,ave,percentage,drama_id
0,cast,松たか子,301,itsumo,いつもふたりで,http://www.fujitv.co.jp/itsumo/,月曜21：00〜21：54 （全11回）,Mon,21:00,21:54,11,11,有名小説家になりたいという夢を持つ谷町瑞穂（松たか子）は応募した新人文学賞を受賞し、遂にその...,フジテレビ,"[1/6, 1/13, 1/20, 1/27, 2/3, 2/10, 2/17, 2/24,...",16.17,"[18.20, 16.40, 15.30, 17.30, 16.00, 15.10, 14....",301_itsumo_いつもふたりで
1,cast,坂口憲二,301,itsumo,いつもふたりで,http://www.fujitv.co.jp/itsumo/,月曜21：00〜21：54 （全11回）,Mon,21:00,21:54,11,11,有名小説家になりたいという夢を持つ谷町瑞穂（松たか子）は応募した新人文学賞を受賞し、遂にその...,フジテレビ,"[1/6, 1/13, 1/20, 1/27, 2/3, 2/10, 2/17, 2/24,...",16.17,"[18.20, 16.40, 15.30, 17.30, 16.00, 15.10, 14....",301_itsumo_いつもふたりで
2,cast,柏原崇,301,itsumo,いつもふたりで,http://www.fujitv.co.jp/itsumo/,月曜21：00〜21：54 （全11回）,Mon,21:00,21:54,11,11,有名小説家になりたいという夢を持つ谷町瑞穂（松たか子）は応募した新人文学賞を受賞し、遂にその...,フジテレビ,"[1/6, 1/13, 1/20, 1/27, 2/3, 2/10, 2/17, 2/24,...",16.17,"[18.20, 16.40, 15.30, 17.30, 16.00, 15.10, 14....",301_itsumo_いつもふたりで
3,cast,長谷川京子,301,itsumo,いつもふたりで,http://www.fujitv.co.jp/itsumo/,月曜21：00〜21：54 （全11回）,Mon,21:00,21:54,11,11,有名小説家になりたいという夢を持つ谷町瑞穂（松たか子）は応募した新人文学賞を受賞し、遂にその...,フジテレビ,"[1/6, 1/13, 1/20, 1/27, 2/3, 2/10, 2/17, 2/24,...",16.17,"[18.20, 16.40, 15.30, 17.30, 16.00, 15.10, 14....",301_itsumo_いつもふたりで
4,cast,平山綾,301,itsumo,いつもふたりで,http://www.fujitv.co.jp/itsumo/,月曜21：00〜21：54 （全11回）,Mon,21:00,21:54,11,11,有名小説家になりたいという夢を持つ谷町瑞穂（松たか子）は応募した新人文学賞を受賞し、遂にその...,フジテレビ,"[1/6, 1/13, 1/20, 1/27, 2/3, 2/10, 2/17, 2/24,...",16.17,"[18.20, 16.40, 15.30, 17.30, 16.00, 15.10, 14....",301_itsumo_いつもふたりで
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,prod,清弘誠,310,suekko,末っ子長男姉三人,http://www.tbs.co.jp/suekko/,日曜21：00〜21：54 （全10回）,Sun,21:00,21:54,10,10,NO_SYNOPSIS,TBS,"[10/12, 10/19, 10/26, 11/2, 11/16, 11/23, 11/3...",14.39,"[18.60, 16.40, 12.80, 13.80, 13.20, 11.30, 12....",310_suekko_末っ子長男姉三人
9,prod,金子文紀,310,suekko,末っ子長男姉三人,http://www.tbs.co.jp/suekko/,日曜21：00〜21：54 （全10回）,Sun,21:00,21:54,10,10,NO_SYNOPSIS,TBS,"[10/12, 10/19, 10/26, 11/2, 11/16, 11/23, 11/3...",14.39,"[18.60, 16.40, 12.80, 13.80, 13.20, 11.30, 12....",310_suekko_末っ子長男姉三人
10,prod,高成麻畝子,310,suekko,末っ子長男姉三人,http://www.tbs.co.jp/suekko/,日曜21：00〜21：54 （全10回）,Sun,21:00,21:54,10,10,NO_SYNOPSIS,TBS,"[10/12, 10/19, 10/26, 11/2, 11/16, 11/23, 11/3...",14.39,"[18.60, 16.40, 12.80, 13.80, 13.20, 11.30, 12....",310_suekko_末っ子長男姉三人
11,prod,吉田紀子,310,suekko,末っ子長男姉三人,http://www.tbs.co.jp/suekko/,日曜21：00〜21：54 （全10回）,Sun,21:00,21:54,10,10,NO_SYNOPSIS,TBS,"[10/12, 10/19, 10/26, 11/2, 11/16, 11/23, 11/3...",14.39,"[18.60, 16.40, 12.80, 13.80, 13.20, 11.30, 12....",310_suekko_末っ子長男姉三人


In [8]:
unq_actors = pd.Series(df['attr_value'].value_counts().index)
unq_actors

0           要潤
1        小日向文世
2         生瀬勝久
3       佐々木蔵之介
4         遠藤憲一
         ...  
5011        初音
5012     眞野あずさ
5013     松岡璃奈子
5014        夏緑
5015      内田康夫
Name: attr_value, Length: 5016, dtype: object

In [9]:
df.groupby('drama_id').ngroups

1036

In [10]:
unq_actors = pd.Series(df['attr_value'].value_counts().index)
unq_act_role = df.dropna()[['attr_name', 'attr_value']].drop_duplicates()
unq_act_role_series = unq_act_role['attr_value'] + unq_act_role['attr_name']
drama2info = defaultdict(list)

grouped = df.groupby('drama_id')
for name, group in tqdm(grouped):
    delete_show = True
    delete_show2 = True
    if "後日更新。。" in group['attr_value'].unique():
        continue
    for cast_role in ['cast']:
        if cast_role in list(group.dropna()['attr_name']):
            delete_show = False
            break
    for cast_role in ['prod']:
        if cast_role in list(group.dropna()['attr_name']):
            delete_show2 = False
            break
    if delete_show:
        continue
    if delete_show2:
        continue

    drama2info['drama_id'].append(name)
    drama2info['aname'].append(name.split("_")[1])
    synopsis_info = group['synopsis'].drop_duplicates()
    assert len(synopsis_info) == 1
    drama2info['synopsis'].append(synopsis_info[0])

    group_act_roles = group['attr_value'] + group['attr_name']
    for actor_id, is_present in zip(
        unq_act_role_series, unq_act_role_series.isin(group_act_roles)):
        if is_present:
            drama2info[actor_id].append(1)
        else:
            drama2info[actor_id].append(0)

  0%|          | 0/1036 [00:00<?, ?it/s]

100%|██████████| 1036/1036 [00:02<00:00, 404.40it/s]


In [11]:
actor_roles_df = pd.DataFrame(drama2info)
print(actor_roles_df.shape)

(1010, 5074)


In [16]:
actor_roles_df = actor_roles_df[actor_roles_df['synopsis'] != ""]
actor_roles_df = actor_roles_df[actor_roles_df['synopsis'] != "NOMATCH"]
actor_roles_df = actor_roles_df[actor_roles_df['synopsis'] != "NO_SYNOPSIS"]
actor_roles_df = actor_roles_df[actor_roles_df['synopsis'] != "NO_NAME_MATCH"]
actor_roles_df = actor_roles_df[actor_roles_df['synopsis'] != "。"]
actor_roles_df = actor_roles_df[actor_roles_df['synopsis'] != "---&gt;"]
print(actor_roles_df.shape)

(601, 5074)


In [17]:
def filter_columns_by_ones_presence(df, min_ones):
    numeric_cols = df.select_dtypes(include=['int', 'float', 'bool']).columns
    filtered_cols = numeric_cols[(df[numeric_cols].sum(axis=0) >= min_ones)]
    excluded_cols = df.columns.drop(filtered_cols)

    return df[['drama_id', 'aname', 'synopsis'] + list(filtered_cols)]



print(actor_roles_df.shape)
actor_roles_df = filter_columns_by_ones_presence(actor_roles_df, min_ones=1)
actor_roles_df.shape

(601, 5074)


(601, 3861)

In [18]:
principal_columns = actor_roles_df.columns[actor_roles_df.columns.str.contains('prod')]
actor_roles_df = actor_roles_df.loc[~(actor_roles_df[principal_columns] == 0).all(axis=1)]
principal_columns = actor_roles_df.columns[actor_roles_df.columns.str.contains('cast')]
actor_roles_df = actor_roles_df.loc[~(actor_roles_df[principal_columns] == 0).all(axis=1)]

In [19]:
actor_roles_df

,drama_id,aname,synopsis,松たか子cast,坂口憲二cast,柏原崇cast,長谷川京子cast,平山綾cast,瑛太cast,佐藤仁美cast,...,末澤誠也cast,安藤ニコcast,「知ってるワイフ」prod,羽村仁成cast,勝村周一朗cast,長州力cast,勝野逸未prod,まなべゆきこprod,吉見一豊cast,中島啓介prod
2,1001_fumouchitai_不毛地帯,fumouchitai,主人公の'''壹岐正'''（いきただし）は陸軍士官学校を首席で卒業したエリート中佐で、大本営...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1001_mygirl_マイガール,mygirl,済州島で観光ガイドをしているチュ・ユリン（イ・ダヘ）をトラブルから助けてあげたプレイボーイの...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1001_ohitorisama_おひとりさま,ohitorisama,東京都内で名門の女子高等学校に教師として勤務する秋山里美（観月ありさ）は33歳で独身、何でも...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1001_real-clothes_リアル・クローズ,real-clothes,越前屋百貨店のふとん売場で働く'''天野絹恵'''は、ある日、百貨店の花形である婦人服売場へ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,1001_samurai_サムライ・ハイスクール,samurai,学校でも家庭でも冴えない草食系高校生・望月小太郎（三浦春馬）。ある日、レポート作成のために立...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,910_otomen_オトメン（乙男）,otomen,柔道・空手の段を持ち、剣道で全国制覇を成し遂げている剣道部主将・正宗飛鳥は、「男の中の男」と...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1006,910_sarulock_猿ロック,sarulock,鍵屋「猿丸ロックサービス」の一人息子・猿丸耶太郎（サル）は「俺に開けられない鍵は無い」と豪語...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1007,910_thespy_華麗なるスパイ,thespy,13件の詐欺容疑で逮捕され、30年の懲役刑を言い渡され服役していた詐欺師・鎧井京介は、日本政...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1008,910_tonarino09_となりの芝生,tonarino09,高平知子はごく普通のサラリーマン家庭の専業主婦であった。夫は次男なので姑の苦労はなく、家は狭...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
def custom_to_datetime(date):
    if date[:2] == '24':
        return pd.to_datetime("20100101"+ "00" + date[2:], format = '%Y%m%d%H:%M') + pd.Timedelta(days=1)
    else:
        return pd.to_datetime("20100101"+date, format = '%Y%m%d%H:%M')

meta_df = df.groupby('drama_id').first().reset_index(drop=False)[
    ['season','drama_id', 'station', 'day', 'startTime', 'endTime', 'slot']]
meta_df['startTime'] = meta_df['startTime'].apply(custom_to_datetime)
meta_df['endTime'] = meta_df['endTime'].apply(custom_to_datetime)
meta_df['length'] = (meta_df['endTime'] - meta_df['startTime']).dt.total_seconds() / 60

def round_to_hour(time_str):
    time = pd.to_datetime(time_str)
    return time.strftime('%I %p').lstrip('0').lower()

meta_df['startTime'] = meta_df['startTime'].apply(round_to_hour)
meta_df['endTime'] = meta_df['endTime'].apply(round_to_hour)

duration_categories = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.2, 1.5, 1.7, 2.0]
meta_df['duration_hours'] = meta_df['length'] / 60
meta_df['length'] = meta_df['duration_hours'].apply(lambda x: min(duration_categories, key=lambda y: abs(y-x)))
meta_df = meta_df.drop(columns=['duration_hours'])

meta_df['slot'] =  meta_df['slot'].str.split('：', n=1).str[0]
meta_df['season'] = meta_df['season'].str[-2:]
df_one_hot = pd.get_dummies(meta_df, columns=['station'],dtype=int)
df_one_hot = pd.get_dummies(df_one_hot, columns=['season'],dtype=int)
df_one_hot = pd.get_dummies(df_one_hot, columns=['day'],dtype=int)
df_one_hot = pd.get_dummies(df_one_hot, columns=['slot'],dtype=int)
df_one_hot = pd.get_dummies(df_one_hot, columns=['startTime'],dtype=int)
df_one_hot = pd.get_dummies(df_one_hot, columns=['endTime'],dtype=int)
df_one_hot = pd.get_dummies(df_one_hot, columns=['length'],dtype=int)

df_one_hot.rename(columns=lambda x: x if x == 'drama_id' else x + "meta", inplace=True)
df_one_hot

,drama_id,station_TBSmeta,station_テレビ朝日meta,station_フジテレビmeta,station_日本テレビmeta,season_01meta,season_04meta,season_07meta,season_10meta,day_Frimeta,...,endTime_10 pmmeta,endTime_11 pmmeta,endTime_12 ammeta,endTime_8 pmmeta,endTime_9 pmmeta,length_0.7meta,length_0.8meta,length_0.9meta,length_1.0meta,length_1.2meta
0,1001_asami09_浅見光彦〜最終章〜,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,1001_bouchou_傍聴マニア09〜裁判長!ここは懲役4年でどうすか〜,0,0,0,1,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
2,1001_fumouchitai_不毛地帯,0,0,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
3,1001_gyne_ギネ 産婦人科の女たち,0,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
4,1001_jin_JIN -仁-,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,910_otomen_オトメン（乙男）,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
1032,910_sarulock_猿ロック,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0
1033,910_thespy_華麗なるスパイ,0,0,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
1034,910_tonarino09_となりの芝生,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0


In [21]:
episode_df = df.groupby('drama_id').first().reset_index(drop=False)[
    ['drama_id', 'season', 'episodes', 'episodesRev', 'date',
     'ave', 'percentage', 'day', 'station', 'slot']]

episode_df['percentage'] = episode_df['percentage']\
    .apply(lambda x : np.nanmean(np.array(x, dtype=np.float32)))

def fix_season(name):
    if len(name) == 3:
        return f"0{name}"
    else:
        return name

def get_two_year_period(row):
    year = row['year']
    month = row['month']

    if 2003 <= year <= 2005:
        period = "2003-2005"
    elif 2006 <= year <= 2008:
        period = "2006-2008"
    elif 2009 <= year <= 2011:
        period = "2009-2011"
    elif 2012 <= year <= 2014:
        period = "2012-2014"
    elif 2015 <= year <= 2017:
        period = "2015-2017"
    elif 2018 <= year <= 2021:
        period = "2018-2021"
    else:
        print(year)
        raise ValueError("ok")

    return f"{period}"

episode_df['season'] = episode_df['season'].astype(str)
episode_df['season'] = episode_df['season'].apply(fix_season)
episode_df['year'] = episode_df['season'].str[:2].astype(int)
episode_df['month'] = episode_df['season'].str[2:]
episode_df['group'] = False
episode_df['year'] = episode_df['year'].apply(lambda x: x + 2000 if x < 100 else x)

episode_df['datetime'] = pd.to_datetime(
    episode_df['year'].astype(str) + episode_df['month'].astype(str), format='%Y%m')
episode_df['two_year_period'] = episode_df.apply(get_two_year_period, axis=1)

sub_df_lis = []
for (stt), agg_med in episode_df.groupby(by=['year'])['ave'].median().items():
    sub_df = episode_df[(episode_df['year'] == stt)].copy()
    sub_df['traction'] = sub_df['ave'] >= agg_med
    sub_df_lis.append(sub_df)
episode_df = pd.concat(sub_df_lis)
episode_df = episode_df.drop(columns=['two_year_period'])

episode_df['early_end'] =  (episode_df['episodesRev'] - episode_df['episodes']) < 0
episode_df['slot'] =  episode_df['slot'].str.split('：', n=1).str[0]
episode_df['stationslot'] =  episode_df['station'] + "_" + episode_df['slot']
episode_df

,drama_id,season,episodes,episodesRev,date,ave,percentage,day,station,slot,year,month,group,datetime,traction,early_end,stationslot
665,301_1ro_刑事★イチロー,0301,10,9,"[1/15, 1/22, 1/29, 2/5, 2/12, 2/19, 2/26, 3/5,...",5.20,5.200000,Wed,TBS,水曜22,2003,01,False,2003-01-01,False,True,TBS_水曜22
666,301_bb_美女か野獣,0301,11,11,"[1/9, 1/16, 1/23, 1/30, 2/6, 2/13, 2/20, 2/27,...",18.45,18.454544,Thu,フジテレビ,木曜22,2003,01,False,2003-01-01,True,False,フジテレビ_木曜22
667,301_bokuiki_僕の生きる道,0301,11,11,"[1/7, 1/14, 1/21, 1/28, 2/4, 2/11, 2/18, 2/25,...",15.46,15.463637,Tue,フジテレビ,火曜22,2003,01,False,2003-01-01,True,False,フジテレビ_火曜22
668,301_cha-han_熱烈的中華飯店,0301,11,10,"[1/8, 1/15, 1/22, 1/29, 2/5, 2/12, 2/19, 2/26,...",8.86,8.860000,Wed,フジテレビ,水曜21,2003,01,False,2003-01-01,False,True,フジテレビ_水曜21
669,301_goodluck_GOOD LUCK!!,0301,10,10,"[1/19, 1/26, 2/2, 2/9, 2/16, 2/23, 3/2, 3/9, 3...",30.41,30.410000,Sun,TBS,日曜21,2003,01,False,2003-01-01,True,False,TBS_日曜21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,2110_kazokuboshuu_#家族募集します,2110,9,9,"[7/9, 7/16, 8/13, 8/20, 8/27, 9/3, 9/10, 9/17,...",7.02,7.000000,Fri,TBS,金曜22,2021,10,False,2021-10-01,False,False,TBS_金曜22
661,2110_kintori2021_緊急取調室,2110,9,9,"[7/8, 7/15, 7/22, 8/12, 8/19, 8/29, 9/2, 9/9, ...",12.18,12.088889,Thu,テレビ朝日,木曜21,2021,10,False,2021-10-01,True,False,テレビ朝日_木曜21
662,2110_oshinooujisama_推しの王子様,2110,11,11,"[7/15, 7/22, 7/29, 8/5, 8/12, 8/19, 8/26, 9/2,...",4.92,4.890909,Thu,フジテレビ,木曜22,2021,10,False,2021-10-01,False,False,フジテレビ_木曜22
663,2110_promise_cinderella_プロミス・シンデレラ,2110,10,10,"[7/13, 7/20, 7/27, 8/3, 8/10, 8/17, 8/24, 8/31...",7.92,7.920000,Tue,TBS,火曜22,2021,10,False,2021-10-01,False,False,TBS_火曜22


In [23]:
actor_roles_df2 = actor_roles_df.merge(df_one_hot, left_on='drama_id', right_on='drama_id')
full_df = actor_roles_df2.merge(episode_df, left_on='drama_id', right_on='drama_id')
full_df

,drama_id,aname,synopsis,松たか子cast,坂口憲二cast,柏原崇cast,長谷川京子cast,平山綾cast,瑛太cast,佐藤仁美cast,...,day,station,slot,year,month,group,datetime,traction,early_end,stationslot
0,1001_fumouchitai_不毛地帯,fumouchitai,主人公の'''壹岐正'''（いきただし）は陸軍士官学校を首席で卒業したエリート中佐で、大本営...,0,0,0,0,0,0,0,...,Thu,フジテレビ,木曜22,2010,01,False,2010-01-01,True,False,フジテレビ_木曜22
1,1001_mygirl_マイガール,mygirl,済州島で観光ガイドをしているチュ・ユリン（イ・ダヘ）をトラブルから助けてあげたプレイボーイの...,0,0,0,0,0,0,0,...,Fri,テレビ朝日,金曜23,2010,01,False,2010-01-01,False,False,テレビ朝日_金曜23
2,1001_ohitorisama_おひとりさま,ohitorisama,東京都内で名門の女子高等学校に教師として勤務する秋山里美（観月ありさ）は33歳で独身、何でも...,0,0,0,0,0,0,0,...,Fri,TBS,金曜22,2010,01,False,2010-01-01,False,False,TBS_金曜22
3,1001_real-clothes_リアル・クローズ,real-clothes,越前屋百貨店のふとん売場で働く'''天野絹恵'''は、ある日、百貨店の花形である婦人服売場へ...,0,0,0,0,0,0,0,...,Tue,フジテレビ,火曜22,2010,01,False,2010-01-01,False,False,フジテレビ_火曜22
4,1001_samurai_サムライ・ハイスクール,samurai,学校でも家庭でも冴えない草食系高校生・望月小太郎（三浦春馬）。ある日、レポート作成のために立...,0,0,0,0,0,0,0,...,Sat,日本テレビ,土曜21,2010,01,False,2010-01-01,True,False,日本テレビ_土曜21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,910_otomen_オトメン（乙男）,otomen,柔道・空手の段を持ち、剣道で全国制覇を成し遂げている剣道部主将・正宗飛鳥は、「男の中の男」と...,0,0,0,0,0,0,0,...,Sat,フジテレビ,土曜23,2009,10,False,2009-10-01,False,False,フジテレビ_土曜23
597,910_sarulock_猿ロック,sarulock,鍵屋「猿丸ロックサービス」の一人息子・猿丸耶太郎（サル）は「俺に開けられない鍵は無い」と豪語...,0,0,0,0,0,0,0,...,Thu,日本テレビ,木曜23,2009,10,False,2009-10-01,False,False,日本テレビ_木曜23
598,910_thespy_華麗なるスパイ,thespy,13件の詐欺容疑で逮捕され、30年の懲役刑を言い渡され服役していた詐欺師・鎧井京介は、日本政...,0,0,0,0,0,0,0,...,Sat,日本テレビ,土曜21,2009,10,False,2009-10-01,True,False,日本テレビ_土曜21
599,910_tonarino09_となりの芝生,tonarino09,高平知子はごく普通のサラリーマン家庭の専業主婦であった。夫は次男なので姑の苦労はなく、家は狭...,0,0,0,0,0,0,0,...,Wed,TBS,水曜21,2009,10,False,2009-10-01,False,False,TBS_水曜21


In [24]:
print(full_df['traction'].value_counts())
full_df['traction'].value_counts()/len(full_df)

traction
False    309
True     292
Name: count, dtype: int64


traction
False    0.514143
True     0.485857
Name: count, dtype: float64

In [25]:
print(full_df['early_end'].value_counts())
full_df['early_end'].value_counts()/len(full_df)

early_end
False    575
True      26
Name: count, dtype: int64


early_end
False    0.956739
True     0.043261
Name: count, dtype: float64

In [29]:
i = full_df[(
            (full_df.drama_id == '2104_24japan_2021_24 JAPAN') |
             (full_df.drama_id == '2104_asagao2_2021_監察医 朝顔') |
            (full_df.drama_id == '401_ashiten_あした天気になあれ。') |
            (full_df.drama_id == '401_beginner_ビギナー') |
            (full_df.drama_id == '401_et_エ・アロール') |
            (full_df.drama_id == '1004_fumouchitai_不毛地帯') |
            (full_df.drama_id == '401_hakoiri_ハコイリムスメ！') |
             (full_df.drama_id == '804_kinpachi_3年B組金八先生') |
             (full_df.drama_id == '401_kyo-han_共犯者') |
             (full_df.drama_id == '401_manhattan_マンハッタンラブストーリー') |
             (full_df.drama_id == '401_tonari_あなたの隣に誰かいる') |
             (full_df.drama_id == '401_trick_トリック') |
             (full_df.drama_id == '1804_todomenokiss_トドメの接吻') |
             (full_df.drama_id == "2004_jyoshimuda_女子高生の無駄遣い") |
             (full_df.drama_id == "1410_suikyuyankees_水球ヤンキース") |
             (full_df.drama_id == "1410_tosca_東京スカーレット　警視庁NS係") |
             (full_df.drama_id == "1510_koinaka_恋仲") |
             (full_df.drama_id == "1607_hinoko_火の粉") |
             (full_df.drama_id == "1610_on_ON　異常犯罪捜査官・藤堂比奈子") |
             (full_df.drama_id == "1610_sukinahitogairukoto_好きな人がいること") |
             (full_df.drama_id == "1704_masuyama_増山超能力師事務所") |
             (full_df.drama_id == "1807_lovererun_ラブリラン") |
             (full_df.drama_id == "1810_tantei_探偵が早すぎる") |
             (full_df.drama_id == "1904_QUEEN_スキャンダル専門弁護士　QUEEN") |
             (full_df.drama_id == "1910_danshare_わたし旦那をシェアしてた") |
             (full_df.drama_id == "2101_koiata_この恋あたためますか") |
             (full_df.drama_id == "1707_haha_母になる")
             )].index
print(full_df.shape)
full_df = full_df.drop(i)
print(full_df.shape)

check_cols = [c for c in full_df.columns if c[-4:] == "cast"] + \
             [c for c in full_df.columns if c[-4:] == "prod"]
act_col_not_present = full_df[check_cols].sum(axis=0) == 0
act_col_not_present = act_col_not_present[act_col_not_present == True].index.to_list()

full_df = full_df.drop(columns=act_col_not_present)
full_df.shape

(601, 3929)
(574, 3929)


(574, 3863)

In [31]:
full_df['station'].value_counts()

station
フジテレビ    198
TBS      142
日本テレビ    138
テレビ朝日     96
Name: count, dtype: int64

In [32]:
full_df['traction'].value_counts()/len(full_df)

traction
False    0.506969
True     0.493031
Name: count, dtype: float64

In [75]:
df_grouped = full_df.groupby(['station', 'slot'])\
    .agg(success_rate=('traction', 'sum')).reset_index()
df_grouped['success_rate'] /= full_df.groupby(['station', 'slot']).size().values
df_grouped['success_bin'] = pd.cut(df_grouped['success_rate'], bins=3)

full_df = pd.merge(full_df, df_grouped, on=['station', 'slot'], how='left')
full_df['success_bin'] = full_df['success_bin'].astype(str)
full_df['success_bin'] = full_df['station'] + "_" + full_df['success_bin']
full_df

,drama_id,aname,synopsis,松たか子cast,坂口憲二cast,柏原崇cast,長谷川京子cast,平山綾cast,瑛太cast,佐藤仁美cast,...,slot,year,month,group,datetime,traction,early_end,stationslot,success_rate,success_bin
0,1001_fumouchitai_不毛地帯,fumouchitai,主人公の'''壹岐正'''（いきただし）は陸軍士官学校を首席で卒業したエリート中佐で、大本営...,0,0,0,0,0,0,0,...,木曜22,2010,01,False,2010-01-01,True,False,フジテレビ_木曜22,0.543478,"フジテレビ_(0.303, 0.606]"
1,1001_mygirl_マイガール,mygirl,済州島で観光ガイドをしているチュ・ユリン（イ・ダヘ）をトラブルから助けてあげたプレイボーイの...,0,0,0,0,0,0,0,...,金曜23,2010,01,False,2010-01-01,False,False,テレビ朝日_金曜23,0.076923,"テレビ朝日_(-0.000909, 0.303]"
2,1001_ohitorisama_おひとりさま,ohitorisama,東京都内で名門の女子高等学校に教師として勤務する秋山里美（観月ありさ）は33歳で独身、何でも...,0,0,0,0,0,0,0,...,金曜22,2010,01,False,2010-01-01,False,False,TBS_金曜22,0.522727,"TBS_(0.303, 0.606]"
3,1001_real-clothes_リアル・クローズ,real-clothes,越前屋百貨店のふとん売場で働く'''天野絹恵'''は、ある日、百貨店の花形である婦人服売場へ...,0,0,0,0,0,0,0,...,火曜22,2010,01,False,2010-01-01,False,False,フジテレビ_火曜22,0.578947,"フジテレビ_(0.303, 0.606]"
4,1001_samurai_サムライ・ハイスクール,samurai,学校でも家庭でも冴えない草食系高校生・望月小太郎（三浦春馬）。ある日、レポート作成のために立...,0,0,0,0,0,0,0,...,土曜21,2010,01,False,2010-01-01,True,False,日本テレビ_土曜21,0.540541,"日本テレビ_(0.303, 0.606]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,910_otomen_オトメン（乙男）,otomen,柔道・空手の段を持ち、剣道で全国制覇を成し遂げている剣道部主将・正宗飛鳥は、「男の中の男」と...,0,0,0,0,0,0,0,...,土曜23,2009,10,False,2009-10-01,False,False,フジテレビ_土曜23,0.000000,"フジテレビ_(-0.000909, 0.303]"
570,910_sarulock_猿ロック,sarulock,鍵屋「猿丸ロックサービス」の一人息子・猿丸耶太郎（サル）は「俺に開けられない鍵は無い」と豪語...,0,0,0,0,0,0,0,...,木曜23,2009,10,False,2009-10-01,False,False,日本テレビ_木曜23,0.000000,"日本テレビ_(-0.000909, 0.303]"
571,910_thespy_華麗なるスパイ,thespy,13件の詐欺容疑で逮捕され、30年の懲役刑を言い渡され服役していた詐欺師・鎧井京介は、日本政...,0,0,0,0,0,0,0,...,土曜21,2009,10,False,2009-10-01,True,False,日本テレビ_土曜21,0.540541,"日本テレビ_(0.303, 0.606]"
572,910_tonarino09_となりの芝生,tonarino09,高平知子はごく普通のサラリーマン家庭の専業主婦であった。夫は次男なので姑の苦労はなく、家は狭...,0,0,0,0,0,0,0,...,水曜21,2009,10,False,2009-10-01,False,False,TBS_水曜21,0.000000,"TBS_(-0.000909, 0.303]"


In [34]:
full_df.to_csv("extracted_actor_positions_artv.csv")